In [56]:
import pandas as pd
import numpy as np
from slugify import slugify
pd.set_option('display.max_columns', None)
pd. set_option("display.max_rows", None)
from Levenshtein import distance

## I. Selecionar os dados: determinar quais conjuntos de dados serão utilizados e documentar os motivos de inclusão/exclusão.

In [57]:
df = pd.read_csv("../data/raw/iowa_liquor_train_test_split_sample.csv")

In [58]:
df.head()

,Unnamed: 0,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,2364808,S10798700028,02/26/2013,4347,Jeff's Market / Blue Grass,102 W MAYNE,BLUE GRASS,52726,"102 W MAYNE\nBLUE GRASS 52726\n(41.509114, -90...",82.0,Scott,1011200.0,STRAIGHT BOURBON WHISKIES,461.0,Campari(skyy),22155,Wild Turkey 101 Round,12,375,$6.00,$8.99,1,$8.99,0.38,0.10
1,12139684,INV-06802100005,08/22/2017,3651,Wal-Mart 1491 / Indianola,1500 North Jefferson,Indianola,50125,1500 North Jefferson\nIndianola 50125\n,91.0,WARREN,1012100.0,Canadian Whiskies,260.0,DIAGEO AMERICAS,11298,Crown Royal Canadian Whisky,6,1750,$31.99,$47.99,1,$287.94,1.75,0.46
2,31671,S14182900098,08/28/2013,2576,Hy-Vee Wine and Spirits / Storm Lake,1250 N LAKE ST,STORM LAKE,50588,"1250 N LAKE ST\nSTORM LAKE 50588\n(42.653594, ...",11.0,Buena Vista,1081700.0,DISTILLED SPIRITS SPECIALTY,421.0,"Sazerac Co., Inc.",86688,Firefly Strawberry Moonshine,12,750,$9.88,$14.82,2,$29.64,1.50,0.40
3,1100266,S11292900026,03/26/2013,3756,Main Street Liquors / Manning,316 MAIN ST,MANNING,51455,"316 MAIN ST\nMANNING 51455\n(41.908573, -95.06...",14.0,Carroll,1081390.0,IMPORTED SCHNAPPS,260.0,Diageo Americas,69947,Rumple Minze Peppermint Schnapps Liqueur,12,1000,$15.15,$22.72,1,$22.72,1.00,0.26
4,12120532,INV-06737700004,08/17/2017,4997,Downtown Pantry,"218, 6th Ave #101",Des Moines,50309,"218, 6th Ave #101\nDes Moines 50309\n(41.58568...",77.0,POLK,1022100.0,Mixto Tequila,395.0,PROXIMO,89191,Jose Cuervo Especial Reposado Tequila Mini,12,500,$11.50,$17.25,1,$17.25,0.50,0.13


In [59]:
df = pd.read_csv("../data/raw/iowa_liquor_train_test_split_sample.csv", usecols=[2,10,22,23],encoding='utf-8')

In [60]:
df.columns = [slugify(col, lowercase=True, separator='_') for col in df.columns]
df.columns

Index(['date', 'county', 'sale_dollars', 'volume_sold_liters'], dtype='object')

df.head(1)

### Remoção das colunas as quais não iremos utilizar
#### Vamos deixar somente as colunas data, store_location, state_bottle_retail, salle_dollars, volume_sold_liters, pois vamos realizar regressão linear em dados numéricos

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2518216 entries, 0 to 2518215
Data columns (total 4 columns):
 #   Column              Dtype  
---  ------              -----  
 0   date                object 
 1   county              object 
 2   sale_dollars        object 
 3   volume_sold_liters  float64
dtypes: float64(1), object(3)
memory usage: 76.8+ MB


In [62]:
df.head()

,date,county,sale_dollars,volume_sold_liters
0,02/26/2013,Scott,$8.99,0.38
1,08/22/2017,WARREN,$287.94,1.75
2,08/28/2013,Buena Vista,$29.64,1.50
3,03/26/2013,Carroll,$22.72,1.00
4,08/17/2017,POLK,$17.25,0.50


In [63]:
df.shape

(2518216, 4)

In [64]:
df[df['date'].isna()].shape

(0, 4)

In [65]:
df[df['county'].isna()].shape

(15650, 4)

In [66]:
df[df['sale_dollars'].isna()].shape

(2, 4)

In [67]:
df[df['volume_sold_liters'].isna()].shape

(0, 4)

#### Procurando por mais inconsistências

## II. Limpar dados: Corrigir, imputar ou remover valores erroneamente inseridos nos conjuntos de dados.

In [68]:
df = df.dropna()

In [69]:
df['county']=df['county'].apply(slugify)

#### Transformando a coluna data para formato date

In [70]:
df['date'] = pd.to_datetime(df['date'])

In [71]:
df['county']=df['county'].apply(slugify)

In [72]:
df.head()

,date,county,sale_dollars,volume_sold_liters
0,2013-02-26,scott,$8.99,0.38
1,2017-08-22,warren,$287.94,1.75
2,2013-08-28,buena-vista,$29.64,1.50
3,2013-03-26,carroll,$22.72,1.00
4,2017-08-17,polk,$17.25,0.50


In [73]:
counties = df['county'].unique()

#### Procurando por mais inconsistências

##### condados com nome parecido

In [74]:
for a in counties:
    for b in reversed(counties):
        if distance(a,b)==1:
            print(a)

buena-vista
cerro-gordo
cerro-gord
buena-vist
o-brien
obrien


In [75]:
df['sale_dollars'] = df['sale_dollars'].replace({'\$':''}, regex = True).astype(float)

In [76]:
df['volume_sold_liters'] = df['volume_sold_liters'].astype(float)

In [77]:
df.head()

,date,county,sale_dollars,volume_sold_liters
0,2013-02-26,scott,8.99,0.38
1,2017-08-22,warren,287.94,1.75
2,2013-08-28,buena-vista,29.64,1.50
3,2013-03-26,carroll,22.72,1.00
4,2017-08-17,polk,17.25,0.50


#### Utilizar os dados de 2016, pois é o mais recente e tem dados completos

In [78]:
df = df[df['date'].dt.year == 2016]

In [79]:
df.head()

,date,county,sale_dollars,volume_sold_liters
10,2016-06-22,bremer,70.52,4.0
20,2016-12-16,hamilton,90.00,1.2
21,2016-12-22,scott,7.26,1.2
38,2016-01-19,page,376.32,42.0
39,2016-05-18,johnson,156.96,9.0


## III. Construir dados: derivar novos atributos que serão úteis. Por exemplo, derivar o IMC de alguém a partir da altura e peso.

In [80]:
county_group = df.groupby(by=["county"]).sum()

In [81]:
county_group.head()

,sale_dollars,volume_sold_liters
county,,
adair,78170.39,5304.73
adams,16566.53,1207.21
allamakee,146542.63,9515.25
appanoose,148019.33,9113.52
audubon,24182.22,2095.80


In [82]:
most = county_group.nlargest(5, ['volume_sold_liters'])

In [83]:
most.head()

,sale_dollars,volume_sold_liters
county,,
polk,1.110619e+07,600075.43
linn,4.266498e+06,253583.50
scott,3.319892e+06,178965.03
johnson,2.929709e+06,161005.70
black-hawk,2.786531e+06,154994.32


In [84]:
most.head()

,sale_dollars,volume_sold_liters
county,,
polk,1.110619e+07,600075.43
linn,4.266498e+06,253583.50
scott,3.319892e+06,178965.03
johnson,2.929709e+06,161005.70
black-hawk,2.786531e+06,154994.32


In [85]:
type(most)

pandas.core.frame.DataFrame

## IV. Integrar dados: criar novos conjuntos de dados combinando dados de várias fontes.

In [96]:
pop_df = pd.read_csv("../data/raw/pop_cty_iowa.csv",usecols=['STNAME','CTYNAME','POPESTIMATE2016'])

In [97]:
pop_df['CTYNAME'] = pop_df['CTYNAME'].replace({' County':''}, regex = True).str.lower()

In [98]:
pop_df = pop_df[pop_df['STNAME']=="Iowa"]

In [99]:
pop_df = pop_df.drop('STNAME',axis=1)

In [100]:
pop_df['CTYNAME']=pop_df['CTYNAME'].apply(slugify)

In [101]:
pop_df['CTYNAME'].unique()

array(['iowa', 'adair', 'adams', 'allamakee', 'appanoose', 'audubon',
       'benton', 'black-hawk', 'boone', 'bremer', 'buchanan',
       'buena-vista', 'butler', 'calhoun', 'carroll', 'cass', 'cedar',
       'cerro-gordo', 'cherokee', 'chickasaw', 'clarke', 'clay',
       'clayton', 'clinton', 'crawford', 'dallas', 'davis', 'decatur',
       'delaware', 'des-moines', 'dickinson', 'dubuque', 'emmet',
       'fayette', 'floyd', 'franklin', 'fremont', 'greene', 'grundy',
       'guthrie', 'hamilton', 'hancock', 'hardin', 'harrison', 'henry',
       'howard', 'humboldt', 'ida', 'jackson', 'jasper', 'jefferson',
       'johnson', 'jones', 'keokuk', 'kossuth', 'lee', 'linn', 'louisa',
       'lucas', 'lyon', 'madison', 'mahaska', 'marion', 'marshall',
       'mills', 'mitchell', 'monona', 'monroe', 'montgomery', 'muscatine',
       'o-brien', 'osceola', 'page', 'palo-alto', 'plymouth',
       'pocahontas', 'polk', 'pottawattamie', 'poweshiek', 'ringgold',
       'sac', 'scott', 'shelby', '

In [102]:
#pop_df = pop_df[ (pop_df['CTYNAME']=='polk') | (pop_df['CTYNAME']=='linn') | (pop_df['CTYNAME']=='scott') | (pop_df['CTYNAME']=='johnson') | (pop_df['CTYNAME']=='black-hawk')].groupby(by=["CTYNAME"])

In [119]:
pop_df.rename(columns = {'CTYNAME':'county'}, inplace = True)

In [120]:
df = pd.merge(most, pop_df, how = 'inner', on = 'county')

## V.  Formatar dados: Formatar novamente os dados conforme as necessidades dos modelos.

In [121]:
df.rename(columns = {'POPESTIMATE2016':'pop_estimate_2016'}, inplace = True)

In [122]:
df

,county,sale_dollars,volume_sold_liters,pop_estimate_2016
0,polk,1.110619e+07,600075.43,474277
1,linn,4.266498e+06,253583.50,222188
2,scott,3.319892e+06,178965.03,172135
3,johnson,2.929709e+06,161005.70,146928
4,black-hawk,2.786531e+06,154994.32,133077


In [124]:
df['liters_per_person'] = merged['volume_sold_liters']/merged['pop_estimate_2016']

In [127]:
df['dolars_per_person'] = merged['sale_dollars']/merged['pop_estimate_2016']

In [128]:
df

,county,sale_dollars,volume_sold_liters,pop_estimate_2016,liters_per_person,dolars_per_person
0,polk,1.110619e+07,600075.43,474277,1.265243,23.417090
1,linn,4.266498e+06,253583.50,222188,1.141302,19.202199
2,scott,3.319892e+06,178965.03,172135,1.039678,19.286562
3,johnson,2.929709e+06,161005.70,146928,1.095814,19.939757
4,black-hawk,2.786531e+06,154994.32,133077,1.164697,20.939240


In [129]:
df = pd.get_dummies(df, columns = ['county'])

In [130]:
df

,sale_dollars,volume_sold_liters,pop_estimate_2016,liters_per_person,dolars_per_person,county_black-hawk,county_johnson,county_linn,county_polk,county_scott
0,1.110619e+07,600075.43,474277,1.265243,23.417090,0,0,0,1,0
1,4.266498e+06,253583.50,222188,1.141302,19.202199,0,0,1,0,0
2,3.319892e+06,178965.03,172135,1.039678,19.286562,0,0,0,0,1
3,2.929709e+06,161005.70,146928,1.095814,19.939757,0,1,0,0,0
4,2.786531e+06,154994.32,133077,1.164697,20.939240,1,0,0,0,0
